In [1]:
from life_after_bert.data import load_olmpics_data, MCDataset
from life_after_bert.eval import evaluate_decoder

In [2]:
import numpy as np
import torch
import transformers

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
model_name = "gpt2-large"
task = "oLMpics MLM"
data_path = "../tests/data/oLMpics_age_comparison_dev.jsonl"
num_choices = 2
mask_token = "[MASK]"

In [4]:
model = transformers.AutoModelForCausalLM.from_pretrained(model_name)
model.eval()
model.to(device)
pass  # Get rid of print

In [5]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, mask_token=mask_token)  # Add mask_token to vocabulary
if tokenizer.pad_token == None:
    tokenizer.pad_token = tokenizer.eos_token

questions, choice_lists, answer_ids = load_olmpics_data(data_path, num_choices)
dataset = MCDataset(questions, choice_lists, answer_ids, tokenizer, mask_token)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using pad_token, but it is not set yet.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 503397.02it/s]


In [6]:
all_answers, all_preds = evaluate_decoder(model, tokenizer, task, num_choices, dataset, device)
(np.array(all_answers) == np.array(all_preds)).mean()

Evaluating: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:03<00:00,  9.46it/s]


0.696